In [ ]:
from tapo_plug import tapoPlugApi
import bluepy.btle as btle
import datetime
import argparse
import os

print("Run the temperature and moisture module code!")

### For bluepy Peripheral class
global dataByte
dataByte = 0
class ReadDelegate(btle.DefaultDelegate):
    def handleNotification(self, cHandle, data):
        # Access global variables insid class
        global dataByte
        dataByte = data

blue_address = "FC:45:C3:DC:57:E6"
### FC:45:C3:DC:57:E6 -> Temp/Mois
### FC:45:C3:DC:99:87 -> Light

### Connect to HC-08 bluetooth Module
p = 0
while p == 0:
    try:
        if p != 0:
            continue
        p = btle.Peripheral(blue_address)
        p.withDelegate(ReadDelegate())
    except:
        pass

while True:
    try:
        ### Message 1
        p.waitForNotifications(0)
        # Change byte to int data
        mois = 10*dataByte[0] + dataByte[1]
        temp = 10*dataByte[2] + dataByte[3]
        if (temp <= 0.0 or temp > 70) or (mois <= 0.0 or mois > 100):
            continue
        print(temp, mois)
        
        ### Save data into file
        with open('temperature.txt', 'w') as f:
            f.write(temp)
        with open('moisture.txt', 'w') as f:
            f.write(mois)

        ### Control  
        tempThres = 26
        if temp > tempThres:
            device = {"tapoIp": "192.168.68.156", "tapoEmail": "ntubblab@gmail.com", "tapoPassword": "ntubime405"}
            response = tapoPlugApi.plugOn(device)
            print(response)
        elif temp < tempThres - 3:
            device = {"tapoIp": "192.168.68.156", "tapoEmail": "ntubblab@gmail.com", "tapoPassword": "ntubime405"}
            response = tapoPlugApi.plugOff(device)
            print(response)

        moisThres = 60
        if mois > moisThres:
            device = {"tapoIp": "192.168.68.157", "tapoEmail": "ntubblab@gmail.com", "tapoPassword": "ntubime405"}
            response = tapoPlugApi.plugOn(device)
            print(response)
        if mois < moisThres - 10:
            device = {"tapoIp": "192.168.68.157", "tapoEmail": "ntubblab@gmail.com", "tapoPassword": "ntubime405"}
            response = tapoPlugApi.plugOff(device)
            print(response)
            
    ### Reconnect
    except Exception as e:
        try:
            p = 0
            while p == 0:
                if p != 0:
                    continue
                p = btle.Peripheral(blue_address)
                p.withDelegate(ReadDelegate())
                # Initialize, four variables
                message_flag2 = 0
                temp = None
                mois = None
        except:
            pass
